In [1]:
"""A very simple MNIST classifier.
See extensive documentation at
https://www.tensorflow.org/get_started/mnist/beginners
"""
#from __future__ import absolute_import
#from __future__ import division
#from __future__ import print_function

import argparse
import sys

from tensorflow.examples.tutorials.mnist import input_data

import tensorflow as tf

In [2]:
# Import data
data_dir = 'E:/AI/00/data/Mnist/input_data'
mnist = input_data.read_data_sets(data_dir, one_hot=True)

Extracting E:/AI/00/data/Mnist/input_data\train-images-idx3-ubyte.gz
Extracting E:/AI/00/data/Mnist/input_data\train-labels-idx1-ubyte.gz
Extracting E:/AI/00/data/Mnist/input_data\t10k-images-idx3-ubyte.gz
Extracting E:/AI/00/data/Mnist/input_data\t10k-labels-idx1-ubyte.gz


In [3]:
# Define loss and optimizer
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])
learning_rate = tf.placeholder(tf.float32)

with tf.name_scope('reshape'):
    x_image = tf.reshape(x, [-1, 28, 28, 1])

In [4]:
# First convolutional layer - maps one grayscale image to 32 feature maps.
# output [-1, 28, 28, 32]
with tf.name_scope('conv1'):
    # 卷积核为[5,5], 将1通道转为32通道
    h_conv1 = tf.contrib.slim.conv2d(x_image, 32, [5,5],   
                                     padding='SAME', 
                                     activation_fn=tf.nn.relu)

# Pooling layer - downsamples by 2X.
# output [-1, 14, 14, 32]
with tf.name_scope('pool1'):
    # 最大池化, 核为[2,2], 步长为2
    h_pool1 = tf.contrib.slim.max_pool2d(h_conv1, [2,2],    
                                         stride=2, 
                                         padding='VALID')    

```
# First convolutional layer - maps one grayscale image to 32 feature maps.
# output [-1, 28, 28, 32]
with tf.name_scope('conv1'):
    # 卷积核为[5,5], 将1通道转为32通道
    shape = [5, 5, 1, 32]
    W_conv1 = tf.Variable(tf.truncated_normal(shape, stddev=0.1),
                          collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    shape = [32]
    b_conv1 = tf.Variable(tf.constant(0.1, shape=shape))
    l_conv1 = tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], # 步长为 1
                           padding='SAME') + b_conv1
    h_conv1 = tf.nn.relu(l_conv1)
# Pooling layer - downsamples by 2X.
# output [-1, 14, 14, 32]
with tf.name_scope('pool1'):
    # 最大池化, 核为[2,2], 步长为2 
    h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='VALID')
```

In [5]:
# Second convolutional layer -- maps 32 feature maps to 64.
with tf.name_scope('conv2'):
    h_conv2 = tf.contrib.slim.conv2d(h_pool1, 64, [5,5], 
                                     padding='SAME', 
                                     activation_fn=tf.nn.relu)

# Second pooling layer.
with tf.name_scope('pool2'):
    h_pool2 = tf.contrib.slim.max_pool2d(h_conv2, [2,2], stride=[2, 2], 
                                         padding='VALID')

In [6]:
# Fully connected layer 1 -- after 2 round of downsampling, our 28x28 image
# is down to 7x7x64 feature maps -- maps this to 1024 features.
with tf.name_scope('fc1'):
    h_pool2_flat = tf.contrib.slim.avg_pool2d(h_pool2, 
                                              h_pool2.shape[1:3], 
                                              stride=[1, 1], 
                                              padding='VALID')
    h_fc1 = tf.contrib.slim.conv2d(h_pool2_flat, 1024, [1,1], 
                                   activation_fn=tf.nn.relu)

In [7]:
# Dropout - controls the complexity of the model, prevents co-adaptation of
# features.
with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [8]:
# Map the 1024 features to 10 classes, one for each digit
with tf.name_scope('fc2'):
    y = tf.squeeze(tf.contrib.slim.conv2d(h_fc1_drop, 10, [1,1], activation_fn=None))

In [11]:
# The raw formulation of cross-entropy,
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_, logits=y))

l2_loss = tf.add_n( [tf.nn.l2_loss(w) for w in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)] )
total_loss = cross_entropy + 7e-5*l2_loss

# 优化方法,随机梯度下降
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)
#train_step = tf.train.AdamOptimizer(learning_rate).minimize(total_loss)
#train_step = tf.train.MomentumOptimizer(learning_rate).minimize(total_loss)

# 全部变量初始化
sess = tf.Session()
init_op = tf.global_variables_initializer()
sess.run(init_op)

In [12]:
# Train
for step in range(3000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    lr = 0.01
    _, loss, l2_loss_value, total_loss_value = sess.run(
               [train_step, cross_entropy, l2_loss, total_loss], 
               feed_dict={x: batch_xs, 
                          y_: batch_ys, 
                          learning_rate:lr, 
                          keep_prob:0.5})
  
    if (step+1) % 100 == 0:
        print('step %d, entropy loss: %f, l2_loss: %f, total loss: %f' % 
            (step+1, loss, l2_loss_value, total_loss_value))
    # Test trained model
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        print(sess.run(accuracy, feed_dict={x: batch_xs, 
                                            y_: batch_ys, 
                                            keep_prob:0.5}))
    if (step+1) % 1000 == 0:
        print(sess.run(accuracy, feed_dict={x: mnist.test.images,
                                            y_: mnist.test.labels, 
                                            keep_prob:0.5}))

step 100, entropy loss: 2.300273, l2_loss: 92.467461, total loss: 2.306746
0.12
step 200, entropy loss: 2.295677, l2_loss: 92.457520, total loss: 2.302149
0.14
step 300, entropy loss: 2.294123, l2_loss: 92.452492, total loss: 2.300595
0.23
step 400, entropy loss: 2.301105, l2_loss: 92.455399, total loss: 2.307576
0.15
step 500, entropy loss: 2.293355, l2_loss: 92.462074, total loss: 2.299827
0.19
step 600, entropy loss: 2.293309, l2_loss: 92.474342, total loss: 2.299783
0.24
step 700, entropy loss: 2.286942, l2_loss: 92.492989, total loss: 2.293417
0.26
step 800, entropy loss: 2.283655, l2_loss: 92.517387, total loss: 2.290131
0.19
step 900, entropy loss: 2.288879, l2_loss: 92.554214, total loss: 2.295357
0.15
step 1000, entropy loss: 2.274333, l2_loss: 92.593483, total loss: 2.280814
0.19
0.1934
step 1100, entropy loss: 2.277869, l2_loss: 92.643311, total loss: 2.284354
0.2
step 1200, entropy loss: 2.282281, l2_loss: 92.701950, total loss: 2.288770
0.16
step 1300, entropy loss: 2.2583